In [1]:
import sys
import importlib
from types import SimpleNamespace
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.special import softmax
from joblib import Parallel, delayed
import seaborn as sns

sys.path.append("../src")
sys.path.append("../configs")

sys.argv = ['--config', 'config1']

from models import *
from loss import *
from train import *
from data import *

def gpu_unravel(batch):
        input_dict, target_dict = batch
        input_dict = {k: input_dict[k].cuda() for k in input_dict}
        target_dict = {k: target_dict[k].cuda() for k in target_dict}
        return input_dict, target_dict
    
dict_unravel = gpu_unravel

Using config file config1


In [2]:
pwd

'/home/ubuntu/kaggle/kaggle-landmark-recognition-2020-1st-place/notebooks'

In [14]:
name = "config1"
# pretrained_weights = "../models/config1_ckpt_10.pth"
pretrained_weights = "../models/config1/config1_ckpt_2.pth"

csv = "train"

#このtrainはGLRv2のfull trainだと思う。
# train = pd.read_csv(f"../embeddings/{csv}.csv")
train = pd.read_csv(f"../../input/GLDv2/{csv}.csv")

# train["img_folder"] = "/ssd/kaggle-landmark/input/train/"
train["img_folder"] = "../../input/GLDv2/train/"
# train["img_folder"] = "/home/ubuntu/kaggle/input/GLDv2/train/"
train["target"] = 0

In [4]:
train

,id,url,landmark_id,img_folder,target
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820,/home/ubuntu/kaggle/input/GLDv2/train/,0
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169,/home/ubuntu/kaggle/input/GLDv2/train/,0
2,3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914,/home/ubuntu/kaggle/input/GLDv2/train/,0
3,e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140,/home/ubuntu/kaggle/input/GLDv2/train/,0
4,4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474,/home/ubuntu/kaggle/input/GLDv2/train/,0
...,...,...,...,...,...
4132909,fc0f007893b11ba7,https://upload.wikimedia.org/wikipedia/commons...,172138,/home/ubuntu/kaggle/input/GLDv2/train/,0
4132910,39aad18585867916,https://upload.wikimedia.org/wikipedia/commons...,162860,/home/ubuntu/kaggle/input/GLDv2/train/,0
4132911,fd0725460e4ebbec,https://upload.wikimedia.org/wikipedia/commons...,191243,/home/ubuntu/kaggle/input/GLDv2/train/,0
4132912,73691ae29e24ba19,https://upload.wikimedia.org/wikipedia/commons...,145760,/home/ubuntu/kaggle/input/GLDv2/train/,0


In [5]:
aug = A.Compose([ 
                A.SmallestMaxSize(512),
                A.CenterCrop(always_apply=False, p=1.0, height=512, width=512), 
                ], 
                p=1.0
                )

In [6]:
64*3

192

In [19]:
val_ds = GLRDataset(train, normalization=args.normalization, aug=aug)

batch_size = 256
val_dl = DataLoader(dataset=val_ds,
                    batch_size=batch_size,
                    sampler=SequentialSampler(val_ds), collate_fn=collate_fn, num_workers=40, pin_memory=True)

In [ ]:
img = cv2.imread("/home/ubuntu/kaggle/input/GLDv2/train/"

In [19]:
val_ds.load_one('312777185f4d71f1',"/home/ubuntu/kaggle/input/GLDv2/train/")

array([[[ 28,  67, 124],
        [ 30,  69, 126],
        [ 32,  71, 128],
        ...,
        [ 29,  65, 123],
        [ 27,  63, 121],
        [ 27,  63, 121]],

       [[ 31,  70, 127],
        [ 31,  70, 127],
        [ 32,  71, 128],
        ...,
        [ 27,  63, 121],
        [ 28,  64, 122],
        [ 29,  65, 123]],

       [[ 32,  71, 128],
        [ 32,  71, 128],
        [ 32,  71, 128],
        ...,
        [ 29,  65, 123],
        [ 29,  65, 123],
        [ 30,  66, 124]],

       ...,

       [[ 21,  25,  26],
        [ 23,  27,  28],
        [ 16,  17,  19],
        ...,
        [ 10,  11,  13],
        [ 11,  12,  14],
        [ 13,  14,  16]],

       [[ 24,  28,  31],
        [  9,  13,  14],
        [ 11,  12,  14],
        ...,
        [  7,   7,   9],
        [  7,   7,   9],
        [  8,   8,  10]],

       [[ 29,  33,  36],
        [ 14,  18,  21],
        [ 15,  16,  18],
        ...,
        [ 13,  13,  15],
        [ 15,  15,  17],
        [ 17,  17,  19]]

In [16]:
aug

Compose([
  SmallestMaxSize(always_apply=False, p=1, max_size=512, interpolation=1),
  CenterCrop(always_apply=False, p=1.0, height=512, width=512),
], p=1.0, bbox_params=None, keypoint_params=None, additional_targets={})

In [15]:
val_ds

In [12]:
model = Net(args)
model.eval()
model.cuda()

model.load_state_dict(torch.load(pretrained_weights))

model = nn.DataParallel(model)


In [8]:
pwd

'/home/ubuntu/kaggle/kaggle-landmark-recognition-2020-1st-place/notebooks'

In [ ]:
def get_embeddings(dl, model):
    with torch.no_grad():
        embeddings = np.zeros((len(dl.dataset) , 512))
        total = len(dl)
        for idx, batch in tqdm(enumerate(dl), total=len(dl)):
            input_dict, target_dict = dict_unravel(batch)

            outs = model.forward(input_dict, get_embeddings=True)["embeddings"]

            embeddings[idx*batch_size:idx*batch_size+outs.size(0),:] = outs.detach().cpu().numpy()

    return embeddings
#landmark_id to calss_idは不要か。
embeddings = get_embeddings(val_dl, model)
print(embeddings.shape)
np.save(f"../embeddings/{name}_{csv}_embeddings", embeddings)

  0%|          | 0/16145 [00:00<?, ?it/s]

In [23]:
with torch.no_grad():
    embeddings = np.zeros((len(val_dl.dataset) , 512))
    total = len(val_dl)
    for idx, batch in tqdm(enumerate(val_dl), total=len(val_dl)):
        input_dict, target_dict = dict_unravel(batch)

        outs = model.forward(input_dict, get_embeddings=True)["embeddings"]

        embeddings[idx*batch_size:idx*batch_size+outs.size(0),:] = outs.detach().cpu().numpy()

  0%|          | 0/172205 [00:00<?, ?it/s]

FAIL READING IMG 

KeyboardInterrupt: 

/home/ubuntu/kaggle/input/GLDv2/train/5/f/7/5f7105e2932daa07.jpg


In [9]:
embeddings = np.zeros((len(val_dl.dataset) , 512))
embeddings.shape

(4132914, 512)

In [12]:
idx

NameError: name 'idx' is not defined

In [21]:
for b in val_dl:
    break

In [22]:
b

[{'input': tensor([[[[ 0.1083,  0.1254,  0.1597,  ...,  0.2282,  0.2796,  0.3309],
            [ 0.0912,  0.0912,  0.1426,  ...,  0.2282,  0.2624,  0.3138],
            [ 0.0398,  0.0569,  0.1083,  ...,  0.2111,  0.2453,  0.2796],
            ...,
            [-0.3541, -0.3712, -0.3541,  ..., -0.4397, -0.4397, -0.4226],
            [-0.3369, -0.3541, -0.3369,  ..., -0.4397, -0.4226, -0.3883],
            [-0.3369, -0.3369, -0.3198,  ..., -0.4568, -0.4054, -0.3541]],
  
           [[ 0.2052,  0.2227,  0.2577,  ...,  0.3803,  0.4328,  0.5028],
            [ 0.1877,  0.1877,  0.2402,  ...,  0.3627,  0.4153,  0.4678],
            [ 0.1527,  0.1527,  0.2052,  ...,  0.3627,  0.3978,  0.4328],
            ...,
            [-0.2500, -0.2850, -0.2675,  ..., -0.3025, -0.3025, -0.2850],
            [-0.2500, -0.2675, -0.2500,  ..., -0.3025, -0.2850, -0.2500],
            [-0.2325, -0.2500, -0.2325,  ..., -0.3025, -0.2675, -0.2150]],
  
           [[ 0.1825,  0.2173,  0.2522,  ...,  0.3045,  0.374

In [ ]:
for i, b in tqdm(enumerate()

In [10]:
for idx, batch in tqdm(enumerate(val_dl), total=len(val_dl)):
    input_dict, target_dict = dict_unravel(batch)
    break

  0%|          | 0/8073 [00:00<?, ?it/s]

FAIL READING IMGFAIL READING IMGFAIL READING IMGFAIL READING IMGFAIL READING IMGFAIL READING IMG  FAIL READING IMGFAIL READING IMG /home/ubuntu/kaggle/input/GLDv2/train/1/c/8/1c8b5d5d127f593e.jpgFAIL READING IMG  FAIL READING IMGFAIL READING IMGFAIL READING IMG
FAIL READING IMGFAIL READING IMG/home/ubuntu/kaggle/input/GLDv2/train/1/3/8/138b0b633891c1b2.jpgFAIL READING IMGFAIL READING IMG FAIL READING IMGFAIL READING IMG    FAIL READING IMG/home/ubuntu/kaggle/input/GLDv2/train/0/8/f/08f451a334e99c56.jpg/home/ubuntu/kaggle/input/GLDv2/train/8/2/9/8293687da475385f.jpg     
 FAIL READING IMGFAIL READING IMG/home/ubuntu/kaggle/input/GLDv2/train/d/a/e/dae345259dc0d951.jpg /home/ubuntu/kaggle/input/GLDv2/train/9/e/6/9e6e0aa3336cb197.jpgFAIL READING IMGFAIL READING IMG /home/ubuntu/kaggle/input/GLDv2/train/a/5/c/a5c8e24cf12eb40b.jpg/home/ubuntu/kaggle/input/GLDv2/train/8/a/7/8a753264aa170672.jpg/home/ubuntu/kaggle/input/GLDv2/train/6/9/2/69224bed07493d62.jpg /home/ubuntu/kaggle/input/GLDv2/tra

KeyboardInterrupt: 

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/home/ubuntu/anaconda3/envs/pytorch_p37/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(se

In [ ]:
embeddings.shape

In [30]:
np.save(f"../embeddings/{name}_{csv}_embeddings", embeddings)

In [ ]:
embeddings.shape
np.save(f"../embeddings/{name}_{csv}_embeddings", embeddings)